# Optiver's Magic Wand

This is a brainteaser that was part of Optiver's recruiting process some time ago. The problem was the following:

## Problem

Harry Potter has to cross several signals when he walks from his home to school. Each of these signals operate independently. They alternate every 80 seconds between green light and red light. At each signal, there is a counter display that tells him how long it will be before the current signal light changes. Harry has a magic wand which lets him turn a signal from red to green instantaneously. However, this wand comes with limited battery life, so he can use it only for a specified number of times.

  a) If the total number of signals is 2 and Harry can use his magic wand only once, then what is the expected waiting time at the signals when Harry optimally walks from his home to school?
    
  b) What if the number of signals is 3 and Harry can use his magic wand only once?
   
  c) Can you write a code that takes as inputs the number of signals and the number of times Harry can use his magic wand, and outputs the expected waiting time?


## Solution

Let us first investigate the following question: We start our way home with $m \in \mathbb{N}$ wand charges and arrive at the first of $n \in \mathbb{N}$ signals that we will label signal $n$. If it is green we can proceed without waiting. But if the light is red, and we have to wait $X_n \in [0,80]$ seconds. When is it worth it to use a wand charge?

Since the signals are independent, it depends if the immediate gain from using the wand (we gain $X_n$ seconds that we would have had to wait otherwise) outweights the subsequent costs: Instead of passing the remaining $n-1$ lights with $m$ charges we have to pass them with only $m-1$ charges. Hence the expected traversal time would increase.

This means if we execute wand charges at light $n$ with the aim to improve the expecte total waiting time, we simply compare the expected waiting time for $n-1$ lights and $m$ charges $T_{n-1,m}$ with the time for $n-1$ lights and $m-1$ charges $T_{n-1,m-1}$ and use de wand, if

\begin{align}
  X_n > \Delta_{n,m} = \mathbb{E}[T_{n-1,m-1}] - \mathbb{E}[T_{n-1,m}].
\end{align}

Using $\Delta_{n,m}$ as the in expectation optimal threshold for wand usage we get the following formula for the expected waiting time.

\begin{align}
\mathbb{E}[T_{n,m}] &= \mathbb{E}[T_{n,m}\mathbb{1}_{\{X_n=0\}}] + \mathbb{E}[T_{n,m}\mathbb{1}_{\{0<X_n<\Delta_{n,m}\}}] + \mathbb{E}[T_{n,m}\mathbb{1}_{\{X_n>= \Delta_{n,m}\}}]\\
&= \mathbb{E}[T_{n-1,m}\mathbb{1}_{\{X_n=0\}}] + \mathbb{E}[(X_n+T_{n-1,m})\mathbb{1}_{\{0<X_n<\Delta_{n,m}\}}] + \mathbb{E}[T_{n-1,m-1}\mathbb{1}_{\{X_n>= \Delta_{n,m}\}}]\\
&= \mathbb{E}[T_{n-1,m}]\mathbb{P}(X_n=0)\\
&+ \mathbb{E}[\mathbb{E}[X_n|0<X_n<\Delta_{n,m}]\mathbb{1}_{\{0<X_n<\Delta_{n,m}\}}]+\mathbb{E}[T_{n-1,m}] \mathbb{P}(0<X_n<\Delta_{n,m})\\
&+ \mathbb{E}[T_{n-1,m-1}] \mathbb{P}(X_n>= \Delta_{n,m})\\
&= \frac{1}{2}\mathbb{E}[T_{n-1,m}] + \frac{\Delta_{n,m}}{160}\left(\frac{\Delta_{n,m}}{2}+\mathbb{E}[T_{n-1,m}]\right) + \frac{80-\Delta_{n,m}}{160}\mathbb{E}[T_{n-1,m-1}].
\end{align}

This gives us a recursive pattern where $\mathbb{E}[T_{n,m}]$ deterministically depends on $\mathbb{E}[T_{n-1,m}]$ and $\mathbb{E}[T_{n-1,m-1}]$, with the starting values

\begin{align}
\mathbb{E}[T_{1,0}] &=  20, \\
\mathbb{E}[T_{1,1}] &=  0.
\end{align}

Furhtermore it holds that
\begin{align}
\mathbb{E}[T_{n,0}] &=  20n \quad \text{for all $n \in \mathbb{N}$},\\
\mathbb{E}[T_{n,n}] &=  0 \quad \text{for all $n \in \mathbb{N}$}.
\end{align}


We can easily solve exercise c) and implement this recursion, see the code below.


In [8]:
def wait_time(n_signals: int, m_wands: int, prev_res={}) -> (float, float):
    if str((n_signals, m_wands)) in prev_res:
        return prev_res[str((n_signals, m_wands))]
    if m_wands == 0:
        return n_signals*20
    if n_signals <= m_wands:
        return 0

    s_next_with_1_wand_less = wait_time(n_signals-1, m_wands-1)
    s_next_with_wand = wait_time(n_signals-1, m_wands)
    delta = s_next_with_1_wand_less - s_next_with_wand
    s_n_m = 0.5*s_next_with_wand + delta/160*(delta/2 + s_next_with_wand) + (80-delta)/160*s_next_with_1_wand_less
    prev_res[str((n_signals, m_wands))] = s_n_m
    return round(s_n_m,2)


Specifically for a) and b) we get the following expected waiting times.
For illustrative purposes we give some more values.

In [9]:
print('Lights: 2, Charges: 1, Expected waiting time: ', wait_time(2, 1))  # a)
print('Lights: 3, Charges: 1, Expected waiting time: ', wait_time(3, 1))  # b)
print('Lights: 3, Charges: 2, Expected waiting time: ', wait_time(3, 2))
print('Lights: 4, Charges: 1, Expected waiting time: ', wait_time(4, 1))
print('Lights: 4, Charges: 2, Expected waiting time: ', wait_time(4, 2))
print('Lights: 4, Charges: 3, Expected waiting time: ', wait_time(4, 3))
print('Lights: 5, Charges: 1, Expected waiting time: ', wait_time(5, 1))
print('Lights: 5, Charges: 2, Expected waiting time: ', wait_time(5, 2))
print('Lights: 5, Charges: 3, Expected waiting time: ', wait_time(5, 3))
print('Lights: 5, Charges: 4, Expected waiting time: ', wait_time(5, 4))

Lights: 2, Charges: 1, Expected waiting time:  8.75
Lights: 3, Charges: 1, Expected waiting time:  21.32
Lights: 3, Charges: 2, Expected waiting time:  4.14
Lights: 4, Charges: 1, Expected waiting time:  35.99
Lights: 4, Charges: 2, Expected waiting time:  11.81
Lights: 4, Charges: 3, Expected waiting time:  2.01
Lights: 5, Charges: 1, Expected waiting time:  51.94
Lights: 5, Charges: 2, Expected waiting time:  22.07
Lights: 5, Charges: 3, Expected waiting time:  6.61
Lights: 5, Charges: 4, Expected waiting time:  0.99
